In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def zdt1(x):
    n = len(x)
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / (n - 1)
    f2 = g * (1 - np.sqrt(x[0] / g))
    return f1, f2

def zdt2(x):
    n = len(x)
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / (n - 1)
    f2 = g * (1 - (x[0] / g)**2)
    return f1, f2

In [38]:
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

import pandas as pd
import numpy as np
import numexpr
import os
import torch
import pickle

tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import normalize, unnormalize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood

from botorch.acquisition.multi_objective.monte_carlo import (
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.sampling import sample_simplex

import time
import warnings

from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.hypervolume import Hypervolume
from botorch.utils.multi_objective.pareto import is_non_dominated
print(tkwargs)

{'dtype': torch.float64, 'device': device(type='cpu')}


In [15]:
# ini_constraints = ['x1 + x2 + x3 - ']

# def check_constraints(x):
#     return (x.sum(dim=-1) - 200).view(-1,1)

# def generate_initial_data(n, bounds):
#     # generate training data
#     train_x = draw_sobol_samples(bounds=bounds, n=n, q=1).squeeze(1)
#     train_con = check_constraints(train_x)
    
#     return train_x, train_con

def initialize_model(train_x, train_obj, bounds):
    # define models for objective and constraint
    train_x = normalize(train_x, bounds)
    models = []
    for i in range(train_obj.shape[-1]):
        train_y = train_obj[..., i:i+1]
        # train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
        models.append(
            SingleTaskGP(train_x, train_y, outcome_transform=Standardize(m=1)) 
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model


def optimize_qnehvi(model, bounds, train_x, train_obj, sampler, ref_point, inequality_constraints=None):
    """Optimizes the qNEHVI acquisition function, and returns a new candidate and observation."""
    standard_bounds = torch.zeros(2, train_x.shape[-1], **tkwargs)
    standard_bounds[1] = 1
    train_x = normalize(train_x, bounds)
    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point.tolist(),  # use known reference point
        X_baseline=train_x,
        sampler=sampler,
        prune_baseline=True,
        # define an objective that specifies which outcomes are the objectives
        objective=IdentityMCMultiOutputObjective(outcomes=[i for i in range(train_obj.shape[-1])]),
        # specify that the constraint is on the last outcome
        # constraints=[lambda Z: Z[..., -1]],
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 400},
        sequential=True,
        inequality_constraints=inequality_constraints
    )
    
    return candidates

def pre_by_model(models, x, bounds):
    test_x = normalize(x, bounds)
    results = None
    for i in range(len(models)):
        a = models[i].posterior(test_x).mean.detach().cpu().numpy()
        b = models[i].posterior(test_x).variance.detach().cpu().numpy()
        b=np.sqrt(b)
        c = np.concatenate([a,b], axis=-1)
        if results is None:
            results = c
        else:
            results = np.concatenate([results, c], axis=0)
    return results

In [16]:
def get_new_df(new_x, model_qnehvi, bounds):
    result = pre_by_model(model_qnehvi.models, new_x, bounds)
    new_x = new_x.detach().cpu().numpy()
    select_df1 = pd.DataFrame({'ratio':new_x[:,0],
                               'T1': new_x[:,1],
                               't1': new_x[:,2],
                               'T2': new_x[:,3],
                               't2': new_x[:,4],
                               'T3': new_x[:,5],
                               't3': new_x[:,6],
                               'T4': new_x[:,7],
                               't4': new_x[:,8],})

    select_df1[['pre_strength', 'pre_module', 'pre_elongation']] =  result[:,0].reshape(-1, BATCH_SIZE).T
    select_df1[['pre_strength_bar', 'pre_module_bar', 'pre_elongation_bar']] =  result[:,1].reshape(-1, BATCH_SIZE).T
    
    select_df1 = select_df1[[
        'ratio', 'T1', 't1', 'T2', 't2', 'T3', 't3', 'T4', 't4',
        'pre_strength', 'pre_strength_bar', 'pre_module','pre_module_bar',
        'pre_elongation', 'pre_elongation_bar'        
    ]]
    return select_df1

def get_ehvi(model_qnehvi, ref_point, train_x, train_obj, can):
    qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
    acq_func = qNoisyExpectedHypervolumeImprovement(
            model=model_qnehvi,
            ref_point= ref_point, # current_state['ref_point'].tolist(),  # use known reference point
            X_baseline=train_x,
            sampler=qnehvi_sampler,
            prune_baseline=True,
            # define an objective that specifies which outcomes are the objectives
            objective=IdentityMCMultiOutputObjective(outcomes=[i for i in range(train_obj.shape[-1])]),
            # specify that the constraint is on the last outcome
            # constraints=[lambda Z: Z[..., -1]],
        )
    result = [acq_func(can[i].view(1,-1)).detach().item() for i in range(can.shape[0])]
    return result

In [26]:
path=''
# from benchmark_functions import branin,Currin
# functions=[branin,Currin]

# 输入维度
x_dim = 9

# 目标
inputs = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']
functions = ['zdt2_y1', 'zdt2_y2']

# 随机数确定
seed=0
np.random.seed(seed)

# X的边界
bound=[0,1]
Fun_bounds = [bound]*x_dim

bounds = torch.tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
                       [1., 1., 1., 1., 1., 1., 1., 1., 1.]] , **tkwargs)

BATCH_SIZE = 4
NUM_RESTARTS = 10 
RAW_SAMPLES = 2048 
N_BATCH = 20 
MC_SAMPLES = 2048 

In [23]:
def opt(filename, batch_size, log, output):
    df = pd.read_csv(path + filename + '.csv')
    df = df.sample(frac=1.0)
    train_x = np.array(df[inputs])
    train_x = torch.tensor(train_x, **tkwargs)
    train_obj = torch.tensor(np.array(df[functions]), **tkwargs)
    mll_qnehvi, model_qnehvi = initialize_model(train_x, train_obj, bounds)
    fit_gpytorch_mll(mll_qnehvi)
    
    ref_point = train_obj.mean(axis=0)
    # ref_point = ref_point + train_obj.std(axis=0) * torch.tensor([0, 3.14, -1], **tkwargs)
    hv = Hypervolume(ref_point=ref_point)
    if train_obj.shape[0] > 0:
        pareto_mask = is_non_dominated(train_obj)
        pareto_y = train_obj[pareto_mask]
        # compute hypervolume
        volume = hv.compute(pareto_y)
    else:
        volume = 0.0

    hvs = []
    hvs.append(volume)
    print(hvs)
    
    qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
    candidates = optimize_qnehvi(model_qnehvi, bounds, train_x, train_obj, qnehvi_sampler, ref_point)
    new_x =  unnormalize(candidates.detach(), bounds=bounds)
    
    # new_df = get_new_df(new_x, model_qnehvi, bounds)
    # ehvi = get_ehvi(model_qnehvi, ref_point.tolist(), train_x, train_obj, candidates)
    # new_df['ehvi'] = [ehvi[i] for i in range(len(ehvi))]

    # map_bounds = np.array([[95., 120., 130., 120., 170., 120.|],
    #                        [125., 300., 160., 300., 190., 300.]])

    # tmp_x = np.array(new_df[['Temperature1/C', 'Time1/min', 'Temperature2/C','Time2/min', 'Temperature3/C', 'Time3/min']])
    # tmp_x = (train_x-map_bounds[0,:])/(map_bounds[1,:]-map_bounds[0,:])

    current_state = {'x_dim': x_dim, 'bounds': bounds, 'ref_point': ref_point, 
                      'train_x': train_x, 'train_obj': train_obj, 'hvs': hvs, 'candidates': candidates,
                      'new_x': new_x, 'model_state_dict': model_qnehvi.state_dict(), 'df': df} #,'result_df': new_df}
    
    select_df = pd.DataFrame()
    select_df[inputs] = new_x
    select_df.to_csv('%s.csv'%output, index=False)
    with open('%s.pkl'%log, 'wb') as f:
        pickle.dump(current_state, f)

def converge(filename, output, result):
    df1 = pd.read_csv(output+'.csv')
    x_ = df1[inputs]
    x_inputs = np.array(x_)
    zdt2_values = -np.array([zdt2(x) for x in x_inputs])

    df = pd.read_csv(path + filename + '.csv')
    x = np.array(df[inputs])
    y = np.array(df[functions])
    y_ = zdt2_values

    new_x = np.concatenate([x, x_])
    new_y = np.concatenate([y, y_])

    new_df = pd.DataFrame()
    new_df[inputs] = new_x
    new_df[functions] = new_y
    new_df.to_csv('%s.csv'%result, index=False)

In [19]:
i = 1
filename = 'test_zdt12'
batch_size = 4
log = 'log_%s'%i
output = 'select_%s'%i
opt(filename, batch_size, log, output)
converge(filename, output, result='test_zdt12_%s'%i)

[0.6187559842475041]


In [31]:
i = 2
filename = 'test_zdt12_%s'%(i-1)
batch_size = 4
log = 'log_%s'%i
output = 'select_%s'%i
opt(filename, batch_size, log, output)
converge(filename, output, result='test_zdt12_%s'%i)

[0.5874053006847657]


In [32]:
i = 3
filename = 'test_zdt12_%s'%(i-1)
batch_size = 4
log = 'log_%s'%i
output = 'select_%s'%i
opt(filename, batch_size, log, output)
converge(filename, output, result='test_zdt12_%s'%i)

[0.5953386837366733]


In [33]:
i = 4
filename = 'test_zdt12_%s'%(i-1)
batch_size = 4
log = 'log_%s'%i
output = 'select_%s'%i
opt(filename, batch_size, log, output)
converge(filename, output, result='test_zdt12_%s'%i)

[0.6601301131807672]


In [39]:
i = 5
filename = 'test_zdt12_%s'%(i-1)
batch_size = 4
log = 'log_%s'%i
output = 'select_%s'%i
opt(filename, batch_size, log, output)
converge(filename, output, result='test_zdt12_%s'%i)

[0.7083437905125926]


In [40]:
i = 6
filename = 'test_zdt12_%s'%(i-1)
batch_size = 4
log = 'log_%s'%i
output = 'select_%s'%i
opt(filename, batch_size, log, output)
converge(filename, output, result='test_zdt12_%s'%i)

[0.7075984920623659]
